In [ ]:
pip install plot_keras_history

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 321 kB 3.2 MB/s 
  Created wheel for plot-keras-history: filename=plot_keras_history-1.1.36-py3-none-any.whl size=9236 sha256=1a8debb72833546eb71761d5ea645a8043ee9b1390b56617c26bee89c8057cf0
  Stored in directory: /root/.cache/pip/wheels/ca/73/ed/41022b42c1d9ce60a8a2ebb71f042c7f895673760404d39fd7
  Created wheel for sanitize-ml-labels: filename=sanitize_ml_labels-1.0.43-py3-none-any.whl size=319231 sha256=60590e331b258e5a84813fd2fd1f191886aa2af04b6b9015fb7aa48d5a1c5017
  Stored in directory: /root/.cache/pip/wheels/71/d5/dd/d4893fffb9dcb9494de70478ab9728ae5d105f7d849a90e81a
  Created wheel for compress-json: filename=compress_json-1.0.7-py3-none-any.whl size=5233 sha256=98bb763a982c6a46e4397ef5c831e3bd4b6e10fe49ee803ffeac2836922c6fa6
  Stored in directory: /root/.cache/pip/wheels/6d/7e/13/0e370d719528969d430498ca952417324595d116776840b279
Successful

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import json
import math
import re

from keras.layers import Input, LSTM, Activation, Lambda, Dense, Embedding, Dropout, Bidirectional, GlobalMaxPool1D
from keras import backend as K
from keras.models import Model, Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils import vis_utils


In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive/")

Mounted at /content/MyDrive/


In [ ]:
data = pd.read_csv('/content/MyDrive/MyDrive/Project/train-balanced-sarcasm.csv',sep = ',')
data.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1010827, 10)

In [ ]:
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2.0,-1.0,-1.0,2016-10,16-10-2016 23:55,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4.0,-1.0,-1.0,2016-11,01-11-2016 00:24,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3.0,3.0,0.0,2016-09,22-09-2016 21:45,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8.0,-1.0,-1.0,2016-10,18-10-2016 21:03,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6.0,-1.0,-1.0,2016-12,30-12-2016 17:00,Yep can confirm I saw the tool they use for th...


In [ ]:
data.tail()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
1010822,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2.0,2.0,0.0,2009-04,25-04-2009 00:47,"No one is calling this an engineered pathogen,..."
1010823,1,"whatever you do, don't vote green!",BCHarvey,climate,1.0,1.0,0.0,2009-05,14-05-2009 22:27,In a move typical of their recent do-nothing a...
1010824,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1.0,1.0,0.0,2009-01,11-01-2009 00:22,Screw the Disabled--I've got to get to Church ...
1010825,1,The Slavs got their own country - it is called...,catsi,worldnews,1.0,1.0,0.0,2009-01,23-01-2009 21:12,I've always been unsettled by that. I hear a l...
1010826,1,"values, as in capitalism .. there is good mone...",frogking,politics,2.0,2.0,0.0,2009-01,24-01-2009 06:20,Why do the people who make our laws seem unabl...


In [ ]:
data.drop(['author','subreddit','score','ups','downs','date','created_utc'],axis=1, inplace=True)

In [ ]:
data.shape

(1010827, 3)

In [ ]:
data.head()

,label,comment,parent_comment
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for th...


In [ ]:
pd.isna('label')
pd.isna('comment')
pd.isna('parent_comment')

False

In [ ]:
data["comment_len"] = data["comment"].str.len() 
c = max(data["comment_len"])
print('comment length:',c)

data["parent_comment_len"] = data["parent_comment"].str.len()
pc = max(data["parent_comment_len"])
print('parent_comment length:',pc)

data.head()

comment length: 10000.0
parent_comment length: 32759.0


,label,comment,parent_comment,comment_len,parent_comment_len
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd ...",10.0,80.0
1,0,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...,74.0,134.0
2,0,"They were underdogs earlier today, but since G...",They're favored to win.,121.0,23.0
3,0,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz,60.0,26.0
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for th...,31.0,85.0


In [ ]:
data.drop(['comment_len','parent_comment_len'],axis=1,inplace=True)
data.head()

,label,comment,parent_comment
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",deadass don't kill my buzz
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for th...


In [ ]:
print("Before preprocessing size of data: {}".format(len(data)))
print('Before preprocessing type of data:', type(data))

#remove rows for comment and parent_comment which are not string and for label which are not int
data = data[data.label.apply(lambda x: type(x)==int)]
data = data[data.comment.apply(lambda x: type(x) == str)]
data = data[data.parent_comment.apply(lambda x: type(x) == str)]
print("After preprocessing size of data: {}".format(len(data)))
print('After preprocessing type of data:', type(data))


Before preprocessing size of data: 1010827
Before preprocessing type of data: <class 'pandas.core.frame.DataFrame'>
After preprocessing size of data: 945241
After preprocessing type of data: <class 'pandas.core.frame.DataFrame'>


In [ ]:
mispell_dict = {"ain't": "is not", "cannot": "can not", "aren't": "are not", "can't": "can not", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "this's": "this is", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is", "they'd": "they would",
                "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
                "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "wont": "will not", "won't've": "will not have", "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color',
                'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What',
                'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I',
                'theBest': 'the best', 'howdoes': 'how does', 'Etherium': 'Ethereum',
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what',
                'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

mispell_dict = {k.lower(): v.lower() for k, v in mispell_dict.items()}

In [ ]:
def preprocessing_text(s):
    # making our string lowercase & removing extra spaces
    s = str(s).lower().strip()
    
    # remove contractions.
    s = " ".join([mispell_dict[word] if word in mispell_dict.keys() else word for word in s.split()])
    
    # removing \n
    s = re.sub('\n', '', s)
    
    # put spaces before & after punctuations to make words seprate. Like "king?" to "king", "?".
    s = re.sub(r"([?!,+=—&%\'\";:¿।।।|\(\){}\[\]//])", r" \1 ", s)
    
    # Remove more than 2 continues spaces with 1 space.
    s = re.sub('[ ]{2,}', ' ', s).strip()
    
    return s

In [ ]:
# apply preprocessing_text function
data['comment'] = data['comment'].apply(preprocessing_text)
data['parent_comment'] = data['parent_comment'].apply(preprocessing_text)
data.head()

,label,comment,parent_comment
0,0,nc and nh.,"yeah , i get that argument. at this point , i ..."
1,0,you do know west teams play against west teams...,the blazers and mavericks ( the wests 5 and 6 ...
2,0,"they were underdogs earlier today , but since ...",they are favored to win.
3,0,"this meme is not funny none of the "" new york ...",deadass do not kill my buzz
4,0,i could use one of those tools.,yep can confirm i saw the tool they use for th...


In [ ]:
data["comment_len"] = data["comment"].str.len() 
c = max(data["comment_len"])
print('comment length:',c)

data["parent_comment_len"] = data["parent_comment"].str.len()
pc = max(data["parent_comment_len"])
print('parent_comment length:',pc)

data.head()

comment length: 10004
parent_comment length: 23735


,label,comment,parent_comment,comment_len,parent_comment_len
0,0,nc and nh.,"yeah , i get that argument. at this point , i ...",10,86
1,0,you do know west teams play against west teams...,the blazers and mavericks ( the wests 5 and 6 ...,75,136
2,0,"they were underdogs earlier today , but since ...",they are favored to win.,125,24
3,0,"this meme is not funny none of the "" new york ...",deadass do not kill my buzz,63,27
4,0,i could use one of those tools.,yep can confirm i saw the tool they use for th...,31,85


In [ ]:
data.drop(['comment_len','parent_comment_len'],axis=1,inplace=True)
data.head()

,label,comment,parent_comment
0,0,nc and nh.,"yeah , i get that argument. at this point , i ..."
1,0,you do know west teams play against west teams...,the blazers and mavericks ( the wests 5 and 6 ...
2,0,"they were underdogs earlier today , but since ...",they are favored to win.
3,0,"this meme is not funny none of the "" new york ...",deadass do not kill my buzz
4,0,i could use one of those tools.,yep can confirm i saw the tool they use for th...


In [ ]:
#save prepared data
data.to_csv('/content/MyDrive/MyDrive/Project/train-balanced-sarcasm-preprocessed.csv', index = False, sep = ',')